## Backtesting Candlestick Patterns in Python - Hammer & Hanging Man

Reference: https://greyhoundanalytics.com/blog/backtesting-candlestick-patterns-in-python/

In [8]:
import vectorbt as vbt
from datetime import datetime, timedelta
import talib

In [6]:
end_time = datetime.now()
start_time = end_time - timedelta(days = 5)

In [39]:
data = vbt.YFData.download("BTC-USD", interval="1h", 
                           start=start_time,
                           end=end_time).get()
print(data)

                                   Open          High           Low  \
Datetime                                                              
2023-08-03 14:00:00+00:00  29162.126953  29330.837891  29162.126953   
2023-08-03 15:00:00+00:00  29364.642578  29375.707031  29206.488281   
2023-08-03 16:00:00+00:00  29231.365234  29295.261719  29197.271484   
2023-08-03 17:00:00+00:00  29281.695312  29333.417969  29208.500000   
2023-08-03 18:00:00+00:00  29208.375000  29238.207031  29142.277344   
...                                 ...           ...           ...   
2023-08-08 10:00:00+00:00  29165.710938  29274.009766  29164.953125   
2023-08-08 11:00:00+00:00  29267.353516  29397.947266  29261.664062   
2023-08-08 12:00:00+00:00  29391.802734  29567.000000  29343.025391   
2023-08-08 13:00:00+00:00  29559.564453  29643.375000  29335.861328   
2023-08-08 14:00:00+00:00  29345.833984  29436.732422  29344.980469   

                                  Close     Volume  Dividends  Stock Splits 

In [40]:
hammer = talib.CDLHAMMER(data.Open, data.High, data.Low, data.Close)
hammer[hammer == 100] # fiter out when hammer pattern occur

Datetime
2023-08-06 00:00:00+00:00    100
2023-08-06 10:00:00+00:00    100
dtype: int32

In [41]:
hanging_man = talib.CDLHANGINGMAN(data.Open, data.High, 
                                  data.Low, data.Close)
hanging_man[hanging_man == -100] # filter out when hanging man pattern occur

Series([], Freq: H, dtype: int32)

In [42]:
buys = hammer == 100
sells = hanging_man == -100

pf = vbt.Portfolio.from_signals(data.Close, buys, sells, 
                                fees=0.005, init_cash=1000,
                                size_type="value", size=500)
pf.stats()

Start                         2023-08-03 14:00:00+00:00
End                           2023-08-08 14:00:00+00:00
Period                                  5 days 01:00:00
Start Value                                      1000.0
End Value                                   1004.332786
Total Return [%]                               0.433279
Benchmark Return [%]                           0.355368
Max Gross Exposure [%]                        50.576415
Total Fees Paid                                     2.5
Max Drawdown [%]                               0.625169
Max Drawdown Duration                   1 days 23:00:00
Total Trades                                          1
Total Closed Trades                                   0
Total Open Trades                                     1
Open Trade PnL                                 4.332786
Win Rate [%]                                        NaN
Best Trade [%]                                      NaN
Worst Trade [%]                                 

In [43]:
pf.plot(subplots=["orders","trade_pnl",
                  "cum_returns","cash",
                  "assets","value"]).show()

In [44]:
trade_records = pf.trades.records_readable
trade_records

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,0,0.017219,2023-08-06 00:00:00+00:00,29038.246094,2.5,2023-08-08 14:00:00+00:00,29435.070312,0.0,4.332786,0.008666,Long,Open,0
